<a href="https://colab.research.google.com/github/garoa-startup/.github/blob/main/Recuperando_dados_TB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [127]:
!pip install tb-rest-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [128]:
# ThingsBoard REST API URL
url = "http://demo.thingsboard.io"

# Default Tenant Administrator credentials
username = "projetogaroaufpa@gmail.com"
password = "projetogaroa123"
deviceID = "91a24bc0-dc36-11ec-8e95-c904ddca5a8a"

In [129]:
#conversão de data para UNIX timestamp
import datetime

def converte(data_string):
  data = datetime.datetime.strptime(data_string, "%d/%m/%Y")
  timestamp = int(datetime.datetime.timestamp(data)*1000)
  return timestamp
 
inicioTempo = converte("26/03/2022")
print(inicioTempo)
fimTempo = converte("27/03/2022")
print(fimTempo)

1648252800000
1648339200000


In [130]:
import tb_rest_client.rest_client_ce as tb
import pandas as pd

cli = tb.RestClientCE(base_url=url)

cli.login(username=username, password=password)

#Mostra atributos de um dispositivo
atributos_lista = cli.get_attributes('DEVICE', deviceID, keys='latitude,longitude,altitude,stationCode')
atributosDF = pd.DataFrame(atributos_lista)
display(atributosDF)

""


In [131]:
#Mostra séries temporais
series = cli.get_timeseries('DEVICE',deviceID,
                            keys='umidade,temperatura',limit=10000,
                            agg="AVG",interval=60*60*1000, #média de hora em hora
                            start_ts=inicioTempo,end_ts=fimTempo)
print(series)
series_df = pd.DataFrame(series['Temperatura'])
display(series_df)

{}


KeyError: ignored